In [13]:
# clone our github repo
# %rm -r ./AMP-potency-prediction-EvoGradient/
!git clone https://github.com/MicroResearchLab/AMP-potency-prediction-EvoGradient.git
%cd AMP-potency-prediction-EvoGradient/

rm: cannot remove './AMP-potency-prediction-EvoGradient/': No such file or directory
Cloning into 'AMP-potency-prediction-EvoGradient'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 97 (delta 30), reused 88 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 17.16 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/AMP-potency-prediction-EvoGradient/AMP-potency-prediction-EvoGradient/AMP-potency-prediction-EvoGradient/AMP-potency-prediction-EvoGradient/AMP-potency-prediction-EvoGradient


AMP_classification.py  environment.yaml  model/                    output/
AMP_regression.py      EvoGradient/      otherModels5_2Output.py   overview.jpg
data/                  LICENSE           otherModelsRegression.py  README.md
